This codes are from a book **[Programming in Scala]** written by *Martin Odersky*

## Rducing code duplicatoin

In [1]:
object FileMatcher {
    private def filesHere = (new java.io.File(",")).listFiles
    
    def filesEnding(query: String) = 
        for (file <- filesHere; if file.getName.endsWith(query))
            yield file

    def filesContaining(query: String) = 
        for (file <- filesHere; if file.getName.contains(query))
            yield file

    def filesRegex(query: String) = 
        for (file <- filesHere; if file.getName.matches(query))
            yield file
}

defined object FileMatcher

You can see all of this repititon. It's possible to make a common helper function.

In [2]:
object FileMatcher {
    private def filesHere = (new java.io.File(",")).listFiles
    
    def filesMatching(query: String, 
        matcher: (String, String) => Boolean) = {
            for (file <- filesHere; if matcher(file.getName, query))
            yield file
    }
}

defined object FileMatcher

Now you can simplify three searching methods by having them call the helper method.

In [4]:
object FileMatcher {
    private def filesHere = (new java.io.File(",")).listFiles
    
    def filesMatching(matcher: String => Boolean) = {
            for (file <- filesHere; if matcher(file.getName))
            yield file
    }
    
    def filesEnding(query: String) = 
        filesMatching(_.endsWith(query))
    
    def filesContaining(query: String) = 
        filesMatching(_.contains(query))
    
    def filesRegex(query: String) = 
        filesMatching(_.matches(query))
}

defined object FileMatcher

## Simpliyfying client code

In [5]:
def containsNeg(nums: List[Int]): Boolean = {
    var exists = false
    for (num <- nums)
        if (num < 0)
            exists = true
    exists
}

defined function containsNeg

In [7]:
containsNeg(List(1,2,-1,4))
containsNeg(List(1,2,3,4))

res6_0: Boolean = true
res6_1: Boolean = false

More concise way to define the method

In [8]:
def containsNeg(nums: List[Int]) = nums.exists(_ < 0)

defined function containsNeg

In [9]:
containsNeg(Nil)

res8: Boolean = false

In [10]:
containsNeg(List(-1,2,3))

res9: Boolean = true

In [11]:
// not concise code
def containsOdd(nums: List[Int]): Boolean = {
    var exists = false
    for (num <- nums)
        if (num % 2 == 1)
            exists = true
    exists
}

defined function containsOdd

In [12]:
def continasOdd(nums: List[Int]): Boolean = nums.exists(_%2 == 1)

defined function continasOdd

In [13]:
containsOdd(List(0,2,4))
containsOdd(List(0,2,5))

res12_0: Boolean = false
res12_1: Boolean = true

## Currying

In [15]:
// non-curried function
def plainOldSum(x: Int, y: Int) = x+y
plainOldSum(3,4)

defined function plainOldSum
res14_1: Int = 7

In [17]:
def curriedSum(x:Int)(y:Int) = x+y
curriedSum(1)(2)

defined function curriedSum
res16_1: Int = 3

Here’s a function named first that does in spirit what the first traditional function invocation of curriedSum would do:

In [18]:
def first(x: Int) = (y:Int) => x + y

defined function first

Applying the first function to 1—in other words, invoking the first function and passing in 1—yields the second function:

In [20]:
val second = first(1)
second(2)

second: Int => Int = <function1>
res19_1: Int = 3

There is a way to get an actual reference to curriedSum’s “second” function. You can use the placeholder notation to use curriedSum in a partially applied function expression, like this:

In [21]:
val onePlus = curriedSum(1)_

onePlus: Int => Int = <function1>

In [22]:
onePlus(2)

res21: Int = 3

## Writing new control structures

In [23]:
def twice(op:Double => Double, x:Double) = op(op(x))

defined function twice

In [25]:
twice(_ + 1, 5)

res24: Double = 7.0

In [26]:
import java.io.PrintWriter 
import java.io.File

def withPrintWriter(file: File, op: PrintWriter => Unit) = {
    val writer = new PrintWriter(file)
    try {
        op(writer)
    } finally {
        writer.close()
    }
}

import java.io.PrintWriter 

import java.io.File


defined function withPrintWriter

Given such a method, you can use it like this:
```
withPrintWriter(
  new File("date.txt"),
  writer => writer.println(new java.util.Date)
)
```

One way in which you can make the client code look a bit more like a built-in control structure is to use curly braces instead of parentheses to surround the argument list. In any method invocation in Scala in which you’re passing in exactly one argument, you can opt to use curly braces to surround
the argument instead of parentheses.

In [27]:
def withPrintWriter(file:File)(op: PrintWriter => Unit) = {
    val writer = new PrintWriter(file)
    try {
        op(writer)
    } finally {
        writer.close()
    }
}

defined function withPrintWriter

In this example, the first argument list, which contains one File argument, is written surrounded by parentheses. The second argument list, which contains one function argument, is surrounded by curly braces.
```
val file = new File("date.txt)
withPrintWriter(file) { writer =>
  writer.println(new java.util.Date)
}
```

## By-name parameters

Without using by-name parameters, you could write myAssert like this:

In [32]:
var assertionsEnabled = true
def myAssert(predicate: () => Boolean) = 
  if (assertionsEnabled && !predicate())
    throw new AssertionError

myAssert(() => 5 < 3)

: 

To make a by-name parameter, you give the parameter a type starting with => instead of () =>.

In [43]:
def byNameAssert(predicate: => Boolean) = 
  if (assertionsEnabled && !predicate)
    throw new AssertionError

defined function byNameAssert

In [33]:
byNameAssert(5<3)

: 

In [44]:
def boolAssert(predicate:Boolean) = 
  if (assertionsEnabled && !predicate)
    throw new AssertionError

defined function boolAssert

Because the type of boolAssert’s parameter is Boolean, the expression inside the parentheses in boolAssert(5 > 3) is evaluated before the call to boolAssert. 

By contrast, because the type of byNameAssert’s predicate parameter is => Boolean, the expression inside the parentheses in byNameAssert(5 > 3) is not evaluated before the call to byNameAssert. Instead a function value will be created whose apply method will evaluate 5 > 3, and this function value will be passed to byNameAssert.

The difference between the two approaches, therefore, is that if assertions are disabled, you’ll see any side effects that the expression inside the parentheses may have in boolAssert, but not in byNameAssert.

In [45]:
assertionsEnabled = false
val x = 5

x: Int = 5

In [46]:
boolAssert(x / 0 == 0)

: 

In [47]:
byNameAssert(x / 0 == 0)